In [ ]:
! pip install langchain ctransformers langchain-community pinecone pypdf sentence-transformers

In [ ]:
import os
%pwd

'e:\\MedicalChatbot\\experiment'

In [ ]:
os.chdir('../')
%pwd

'e:\\MedicalChatbot'

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

In [ ]:
def load_pdf(data):
    loader=DirectoryLoader(
        data,
        glob='*.pdf',
        loader_cls=PyPDFLoader
    )
    return loader.load()

In [ ]:
extract_data=load_pdf('data/')
# extract_data

In [ ]:
def text_split(extract_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extract_data)
    return text_chunks


In [ ]:
text_chunks=text_split(extract_data=extract_data)
len(text_chunks)

23701

In [ ]:
def download_embedding():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings=download_embedding()
#embeddings

C:\Users\Kanishka\AppData\Local\Temp\ipykernel_8592\3757407036.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
e:\MedicalChatbot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
query_result=embeddings.embed_query('Hello everyone')
print(len(query_result))

384


In [ ]:
# !pip install langchain_pinecone

In [ ]:
PINECONE_API_KEY='pcsk_2rZ7cP_5vpStJxJjpqumDG3XF2bRcvHBYznisubvX5EgvFZgjG36aDQPR88ynRtmyqCvSQ'


In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medical-chatbot"
index = pc.Index(index_name)

docsearch = PineconeVectorStore.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name,
    pinecone_api_key=PINECONE_API_KEY
)

docsearch


In [ ]:
docsearch=PineconeVectorStore.from_existing_index(index_name,embeddings)

query='What is  HIV? '

docs=docsearch.similarity_search(query,k=3)

print('Answer is : ',docs)

Answer is :  [Document(id='ecc51a45-caae-4b8f-b4bb-8d01bb87653d', metadata={}, page_content='Co-infection with HIV'), Document(id='abe4a538-3a85-402d-aff3-ae693c5ed550', metadata={}, page_content='AIDS is de\ue01fned by the development of speci\ue01fed opportunistic infections, cancers and severe manifestations of HIV itself (CDC category C or WHO stage 4).\nPresenting problems in HIV infection'), Document(id='c971c2b7-b45d-4660-a9ec-19a25acabd4a', metadata={}, page_content='Diagnosis and investigations\nDiagnosing HIV infection')]


In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
prompt=PromptTemplate(
    input_variables=['context','question'],
    template=prompt_template
)

chain_type_kwargs={'prompt':prompt}

In [ ]:
llm=CTransformers(
    model=r'model\llama-2-7b-chat.ggmlv3.q4_0.bin',
    model_type='llama',
    config={
        'max_new_tokens':512,
        'temperature':0.8
    }
)

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    if user_input=='exit':
        break
    result=qa({"query": user_input})
    print('Question :',user_input)
    print("Response : ", result["result"])


C:\Users\Kanishka\AppData\Local\Temp\ipykernel_8592\1496343865.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Question : what are allergies
Response :  Allergy is the inappropriate immune response to a harmless substance, resulting in symptoms such as sneezing, congestion, runny nose, itchy eyes, hives, and swelling. The immune system produces antibodies called IgE, which attach to the offending substance and cause an exaggerated response.
